[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/StatMixedML/XGBoostLSS/blob/master/examples/simulation_example_NegativeBinomial.ipynb)

# Imports

In [10]:
from xgboostlss.model import *
from xgboostlss.distributions.NegativeBinomial import *

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import multiprocessing

# Data

In [2]:
n_cpu = multiprocessing.cpu_count()

def custom_transform(y, constr_val):
    # Apply a custom transformation to restrict y between 0 and constr_val
    transformed_y = np.abs(y)  # Example transformation: logarithmic
    constrained_y = constr_val * transformed_y / np.max(transformed_y)  # Scale to desired range
    int_y = constrained_y.astype(int)
    return int_y

# Generate a custom dataset
X, y = make_regression(n_samples=5000, n_features=10, n_informative=2, random_state=123)

# Apply the custom transformation
y = custom_transform(y, 50)

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(X_train, label=y_train, nthread=n_cpu)
dtest = xgb.DMatrix(X_test, nthread=n_cpu)

# Distribution Selection

In [3]:
# Specifies Negative Binomial distribution with corresponding response functions and option to stabilize Gradient/Hessian. See ?NegativeBinomial for more information.
xgblss = XGBoostLSS(
    NegativeBinomial(stabilization="None",              # Options are "None", "MAD", "L2".
                     response_fn_total_count="relu",    # Function to transform the total_count-parameter, e.g., "exp", "softplus" or "relu".
                     response_fn_probs="sigmoid",       # Function to transform the probs-parameter, e.g., "sigmoid".
                     loss_fn="nll"                      # Loss function. Options are "nll" (negative log-likelihood) or "crps"(continuous ranked probability score).
                    )
)

# Hyper-Parameter Optimization

In [4]:
# Any XGBoost hyperparameter can be tuned, where the structure of the parameter dictionary needs to be as follows:

    # Float/Int sample_type
        # {"param_name": ["sample_type", low, high, log]}
            # sample_type: str, Type of sampling, e.g., "float" or "int"
            # low: int, Lower endpoint of the range of suggested values
            # high: int, Upper endpoint of the range of suggested values
            # log: bool, Flag to sample the value from the log domain or not
        # Example: {"eta": "float", low=1e-5, high=1, log=True]}

    # Categorical sample_type
        # {"param_name": ["sample_type", ["choice1", "choice2", "choice3", "..."]]}
            # sample_type: str, Type of sampling, either "categorical"
            # choice1, choice2, choice3, ...: str, Possible choices for the parameter
        # Example: {"booster": ["categorical", ["gbtree", "dart"]]}

    # For parameters without tunable choice (this is needed if tree_method = "gpu_hist" and gpu_id needs to be specified)
        # {"param_name": ["none", [value]]},
            # param_name: str, Name of the parameter
            # value: int, Value of the parameter
        # Example: {"gpu_id": ["none", [0]]}

# Depending on which parameters are optimized, it might happen that some of them are not used, e.g., when {"booster":  ["categorical", ["gbtree", "gblinear"]]} and {"max_depth": ["int", 1, 10, False]} are
# specified, max_depth is not used when gblinear is sampled, since it has no such argument.

param_dict = {
    "eta":              ["float", {"low": 1e-5,   "high": 1,     "log": True}],
    "max_depth":        ["int",   {"low": 1,      "high": 10,    "log": False}],
    "gamma":            ["float", {"low": 1e-8,   "high": 40,    "log": True}],
    "subsample":        ["float", {"low": 0.2,    "high": 1.0,   "log": False}],
    "colsample_bytree": ["float", {"low": 0.2,    "high": 1.0,   "log": False}],
    "min_child_weight": ["float", {"low": 1e-8,   "high": 500,   "log": True}],
    "booster":          ["categorical", ["gbtree"]],
    # "tree_method":    ["categorical", ["auto", "approx", "hist", "gpu_hist"]],
    # "gpu_id":         ["none", [0]]
}

np.random.seed(123)
opt_param = xgblss.hyper_opt(param_dict,
                             dtrain,
                             num_boost_round=100,        # Number of boosting iterations.
                             nfold=5,                    # Number of cv-folds.
                             early_stopping_rounds=20,   # Number of early-stopping rounds
                             max_minutes=5,              # Time budget in minutes, i.e., stop study after the given number of minutes.
                             n_trials=None,              # The number of trials. If this argument is set to None, there is no limitation on the number of trials.
                             silence=False,              # Controls the verbosity of the trail, i.e., user can silence the outputs of the trail.
                             seed=123,                   # Seed used to generate cv-folds.
                             hp_seed=None                # Seed for random number generator used in the Bayesian hyperparameter search.
                            )

[I 2023-05-18 08:51:54,049] A new study created in memory with name: XGBoostLSS Hyper-Parameter Optimization
Progress bar is experimental (supported from v1.2.0). The interface can change in the future.


   0%|          | 00:00/05:00

[I 2023-05-18 08:52:00,782] Trial 0 finished with value: 2253.7895022000002 and parameters: {'eta': 0.1255364723484603, 'max_depth': 4, 'gamma': 4.514338862336535, 'subsample': 0.3753421259261627, 'colsample_bytree': 0.3776899045118417, 'min_child_weight': 1.207028911491354, 'booster': 'gbtree'}. Best is trial 0 with value: 2253.7895022000002.
[I 2023-05-18 08:52:07,990] Trial 1 finished with value: 2640.5155762000004 and parameters: {'eta': 2.0670646830822567e-05, 'max_depth': 4, 'gamma': 0.05808844939071527, 'subsample': 0.7733659713882497, 'colsample_bytree': 0.464512804425795, 'min_child_weight': 1.7704801900956446e-08, 'booster': 'gbtree'}. Best is trial 0 with value: 2253.7895022000002.
[I 2023-05-18 08:52:16,119] Trial 2 finished with value: 2588.2733884 and parameters: {'eta': 0.0007594823592931716, 'max_depth': 6, 'gamma': 0.372164381471535, 'subsample': 0.8067084910251643, 'colsample_bytree': 0.8897527920318078, 'min_child_weight': 0.4778136689271917, 'booster': 'gbtree'}. Be

# Model Training

In [5]:
np.random.seed(123)

opt_params = opt_param.copy()
n_rounds = opt_params["opt_rounds"]
del opt_params["opt_rounds"]

# Train Model with optimized hyperparameters
xgblss.train(opt_params,
             dtrain,
             num_boost_round=n_rounds
             )

# Prediction

In [6]:
# Set seed for reproducibility
torch.manual_seed(123)

# Number of samples to draw from predicted distribution
n_samples = 1000
quant_sel = [0.05, 0.95] # Quantiles to calculate from predicted distribution

# Sample from predicted distribution
pred_samples = xgblss.predict(dtest,
                              pred_type="samples",
                              n_samples=n_samples,
                              seed=123)

# Calculate quantiles from predicted distribution
pred_quantiles = xgblss.predict(dtest,
                                pred_type="quantiles",
                                n_samples=n_samples,
                                quantiles=quant_sel)

# Returns predicted distributional parameters
pred_params = xgblss.predict(dtest,
                             pred_type="parameters")

In [7]:
pred_samples.head()

,y_sample0,y_sample1,y_sample2,y_sample3,y_sample4,y_sample5,y_sample6,y_sample7,y_sample8,y_sample9,...,y_sample990,y_sample991,y_sample992,y_sample993,y_sample994,y_sample995,y_sample996,y_sample997,y_sample998,y_sample999
0,30,10,24,11,22,19,41,21,27,9,...,46,31,19,35,18,43,18,31,28,40
1,21,28,27,35,11,21,23,22,20,30,...,29,30,21,32,9,33,35,28,9,14
2,13,13,10,25,8,24,23,23,27,27,...,20,16,24,13,13,18,25,18,29,21
3,15,15,8,17,10,9,14,22,14,8,...,8,13,21,11,3,17,19,15,8,6
4,9,5,2,6,6,6,4,1,12,8,...,5,8,4,16,5,5,8,6,3,6


In [8]:
pred_quantiles.head()

,quant_0.05,quant_0.95
0,15,47
1,12,40
2,9,33
3,4,20
4,2,14


In [9]:
pred_params.head()

,total_count,probs
0,12.940989,0.692237
1,12.694552,0.657546
2,11.890333,0.626702
3,10.453554,0.508513
4,10.296977,0.434791
